**Analysis of Rotator Trajectory and Controller**

This notebook is to analyze the performance of rotator trajectory (slew and tracking) and do the debug.

**Import Modules**

In [ ]:
%matplotlib inline
%matplotlib widget
from datetime import datetime
import time
from astropy.time import Time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

from lsst_efd_client import EfdClient

**Declaration of User-defined Functions**

In [ ]:
def retrieve_efd_client(is_summit=True):
    """
    Retrieves a client to EFD.

    Parameters
    ----------
    is_summit : bool, optional
        This notebook is running on the summit or not. If not, the returned object will point
        to the test stand at Tucson.

    Returns
    -------
    EfdClient : The interface object between the Nublado and summit/Tucson EFD.
    """
    efd_name = "summit_efd" if is_summit else "tucson_teststand_efd"        
    return EfdClient(efd_name)

In [ ]:
async def get_data_path_generator(efd_client, time_start, time_end):
    """
    Query and return the rotator demanded and actual path data.

    Parameters
    ----------
    efd_client : lsst_efd_client.efd_helper.EfdClient
        EFD client.
    time_start : astropy.time.core.Time
        Start time.
    time_end : astropy.time.core.Time
        End time.
        
    Returns
    -------
    data : pandas.core.frame.DataFram
        Data of the path generator.
    acceleration : numpy.ndarray
        Acceleration in deg/sec^2.
    jerk : numpy.ndarray
        Jerk in deg/sec^3.
    time_operation : numpy.ndarray
        Operation time.
    """
    # Get the data from EFD
    data = await efd_client.select_time_series(
        "lsst.sal.MTRotator.rotation", 
        fields=["demandPosition", "demandVelocity", "demandAcceleration", "actualPosition", "actualVelocity", "timestamp"], 
        start=time_start, 
        end=time_end)

    # Realign the time origin to 0
    time_operation = np.array(data.timestamp.subtract(data.timestamp[0]))

    # Calculate the acceleration and jerk
    # Note the Simulink model had applied the low-pass filter for the calculation of velocity already 
    acceleration, jerk = calc_acceleration_and_jerk(time_operation, data.actualVelocity)
    
    return data, acceleration, jerk, time_operation

In [ ]:
def calc_acceleration_and_jerk(time, velocity):
    """
    Calculate the acceleration and jerk.

    Parameters
    ----------
    time : numpy.ndarray
        Time in second.
    velocity : pandas.core.series.Series
        velocity (deg/sec).

    Returns
    ----------
    acceleration : numpy.ndarray
        Acceleration in deg/sec^2.
    jerk : numpy.ndarray
        Jerk in deg/sec^3.
    """

    acceleration = np.gradient(velocity, time)
    jerk = np.gradient(acceleration, time)

    return acceleration, jerk

In [ ]:
async def get_data_motors(efd_client, time_start, time_end):
    """
    Get the data of motor.

    Parameters
    ----------
    efd_client : lsst_efd_client.efd_helper.EfdClient
        EFD client.
    time_start : astropy.time.core.Time
        Start time.
    time_end : astropy.time.core.Time
        End time.

    Returns
    -------
    data : pandas.core.frame.DataFram
        Data of the motor.
    """
    # Get the data from EFD
    data = await efd_client.select_time_series(
        "lsst.sal.MTRotator.motors",
        fields=["torque0", "torque1", "current0", "current1", "busVoltage"],
        start=time_start,
        end=time_end)
    
    return data

In [ ]:
def plot_currents(data, time_operation):
    """
    Plot the data of currents.

    Parameters
    -------
    data : pandas.core.frame.DataFram
        Data of the motor.
    time_operation : numpy.ndarray
        Operation time.
    """

    plt.figure()
    plt.plot(time_operation, data.current0, "b")
    plt.plot(time_operation, data.current1, "r")
    plt.title("Currents")
    plt.xlabel("Time (s)")
    plt.ylabel("Current (A)")

In [ ]:
def plot_path_generator(data, acceleration, jerk, time_operation):
    """
    Plot the data of path generator.

    Parameters
    -------
    data : pandas.core.frame.DataFram
        Data of the path generator.
    acceleration : numpy.ndarray
        Acceleration in deg/sec^2.
    jerk : numpy.ndarray
        Jerk in deg/sec^3.
    time_operation : numpy.ndarray
        Operation time.
    """

    plt.figure()
    plt.subplot(5, 1, 1)
    plt.plot(time_operation, data.demandPosition)
    plt.plot(time_operation, data.actualPosition, "x-")
    plt.title("Path Generator")
    plt.ylabel("P")

    plt.subplot(5, 1, 2)
    plt.plot(time_operation, (data.demandPosition - data.actualPosition) * 3600)
    plt.ylabel("dP (arcsec)")

    plt.subplot(5, 1, 3)
    plt.plot(time_operation, data.demandVelocity)
    plt.plot(time_operation, data.actualVelocity, "x-")
    plt.ylabel("V")

    plt.subplot(5, 1, 4)
    plt.plot(time_operation, data.demandAcceleration)
    plt.plot(time_operation, acceleration, "x-")
    plt.ylabel("A")

    plt.subplot(5, 1, 5)
    plt.plot(time_operation, jerk)
    plt.ylabel("J")
    plt.xlabel("Time (s)")
    
    print("Units: P (deg), dP (arcsec), V (deg/s), A (deg/s^2), J (deg/s^3)")

In [ ]:
def analyze_tracking_targets(data, time_operation, list_time_start, list_time_end):
    """
    Analyze the tracked targets and calculate the RMS of the position error.

    Parameters
    -------
    data : pandas.core.frame.DataFram
        Data of the path generator.
    time_operation : numpy.ndarray
        Operation time.
    list_time_start : list
        List of the start time in second.
    list_time_end : list
        List of the end time in second.
    """

    rms_positions = list()
    for time_start, time_end in zip(list_time_start, list_time_end):
        rms_position = analyze_tracking_target_single(data, time_operation, time_start, time_end, show_figure=False)
        rms_positions.append(rms_position)
        print("\n")

    rms_positions_overall = np.sqrt(np.mean(np.array(rms_positions) ** 2))

    digit_after_decimal = 3
    print(f"The overall RMS of position error is {round(rms_positions_overall, digit_after_decimal)} arcsec.")

In [ ]:
def analyze_tracking_target_single(data, time_operation, time_start, time_end, show_figure=True):
    """
    Analyze the single tracked target and calculate the RMS of the position error.

    Parameters
    -------
    data : pandas.core.frame.DataFram
        Data of the path generator.
    time_operation : numpy.ndarray
        Operation time.
    time_start : float
        Start time in second.
    time_end : float
        End time in second.
    show_figure : bool, optional
        Show the figure or not. (the default is True)
    
    Returns
    -------
    rms_position : flost
        RMS of the position error.
    """

    # Get the position data within the time range (in arcsec)
    indices = np.where(np.logical_and(time_operation>=time_start, time_operation<=time_end))[0]
    position_demand = np.array(data.demandPosition[indices]) * 3600
    position_actual = np.array(data.actualPosition[indices]) * 3600
    
    # Analyze the data
    diff_position = position_demand - position_actual
    rms_position = np.sqrt(np.mean(diff_position ** 2))

    # Print the information
    digit_after_decimal = 3
    print(f"Analyze the data between {time_start} sec and {time_end} sec.")
    print(f"There are {len(indices)} points.")
    print(f"The max of position error is {round(np.max((np.abs(diff_position))), digit_after_decimal)} arcsec.")
    print(f"The RMS of position error is {round(rms_position, digit_after_decimal)} arcsec.")

    # Plot the figure
    if show_figure:
        plt.figure()
        plt.title("Position Error")
        plt.plot(time_operation[indices], diff_position)
        plt.xlabel("Time (s)")
        plt.ylabel("dP (arcsec)")
        
    return rms_position

In [ ]:
def calculate_frequency(data, time_operation, time_start, time_end, start_position, tracking_velocity):
    """
    Calculate the oscillation frequency of tracking trajectory: position and velocity.

    Note: This should not happen and Tekniker is debugging now with Rubin's support and help.

    Parameters
    -------
    data : pandas.core.frame.DataFram
        Data of the path generator.
    time_operation : numpy.ndarray
        Operation time.
    time_start : float
        Start time in second.
    time_end : float
        End time in second.
    start_position : float
        Start position of target.
    tracking_velocity : float
        Tracking velocity in degree/second.

    Returns
    -------
    frequency_position : flost
        Oscillation frequency of position in Hz.
    frequency_velocity : flost
        Oscillation frequency of velocity in Hz.
    """
    
    # Get the position data within the time range (in arcsec)
    indices = np.where(np.logical_and(time_operation>=time_start, time_operation<=time_end))[0]
    demand_position = np.array(data.demandPosition[indices]) - start_position - tracking_velocity * time_operation[indices]
    
    dt = time_operation[1] - time_operation[0]
    data_fft_position, frequency_fft_position = calculate_fft(demand_position, dt)
    
    # Get the velocity 
    demand_velocity = np.array(data.demandVelocity[indices])
    data_fft_velocity, frequency_fft_velocity = calculate_fft(demand_velocity, dt)

    num_position = len(demand_position)
    num_velocity = len(demand_velocity)

    plt.figure()
    plt.subplot(2, 1, 1)
    plt.title("Demand position")
    plt.plot(frequency_fft_position, data_fft_position[0:num_position//2], "o-")

    plt.subplot(2, 1, 2)
    plt.title("Demand velocity")
    plt.plot(frequency_fft_velocity, data_fft_velocity[0:num_velocity//2], "o-")
    plt.xlabel("Frequency (Hz)")

In [ ]:
def calculate_fft(data, dt):
    """Calculate FFT.

    Parameters
    ----------
    data : numpy.ndarray
        Data.
    dt : float
        Delta T in seconds.

    Returns
    -------
    data_fft : numpy.ndarray
        FFT data.
    frequency_fft : numpy.ndarray
        Frequency in FFT data (in Hz).
    """
    
    data_fft = fft(data)
    
    num = len(data)
    frequency_fft = fftfreq(num, dt)[:num//2]
    
    return np.abs(data_fft), frequency_fft

In [ ]:
async def get_command_data(efd_client, time_start, time_end):
    """
    Get the command data.

    Parameters
    ----------
    efd_client : lsst_efd_client.efd_helper.EfdClient
        EFD client.
    time_start : astropy.time.core.Time
        Start time.
    time_end : astropy.time.core.Time
        End time.

    Returns
    -------
    data_track : pandas.core.frame.DataFrame
        Data of the track command.
    """

    # Get the data from EFD
    data_track = await efd_client.select_time_series(
        "lsst.sal.MTRotator.command_track", 
        fields=["angle", "velocity", "tai"], 
        start=time_start,
        end=time_end)

    # Add the column of time difference in seconds
    add_column_time_differece(data_track, time_start)

    return data_track

In [ ]:
async def get_event_data(efd_client, time_start, time_end):
    """
    Get the event data.

    Parameters
    ----------
    efd_client : lsst_efd_client.efd_helper.EfdClient
        EFD client.
    time_start : astropy.time.core.Time
        Start time.
    time_end : astropy.time.core.Time
        End time.

    Returns
    -------
    data_controller_state : pandas.core.frame.DataFrame
        Data of the controller state event.
    data_in_position : pandas.core.frame.DataFrame
        Data of the inPosition event.
    data_tracking : pandas.core.frame.DataFrame
        Data of the tracking event.
    """

    # Get the data from EFD
    data_controller_state = await efd_client.select_time_series(
        "lsst.sal.MTRotator.logevent_controllerState", 
        fields=["controllerState", "enabledSubstate"], 
        start=time_start,
        end=time_end)
    
    # The inPosition field is a combination of Flags_moveSuccess and Flags_trackingSuccess in Simulink telemetry
    data_in_position = await efd_client.select_time_series(
        "lsst.sal.MTRotator.logevent_inPosition", 
        fields="inPosition",
        start=time_start,
        end=time_end)
    
    # The tracking field is Flags_trackingSuccess in Simulink telemetry
    # The lost field is Flags_trackingLost in Simulink telemetry
    # The noNewCommand field is Flags_noNewTrackCmdError in Simulink telemetry
    data_tracking = await efd_client.select_time_series(
        "lsst.sal.MTRotator.logevent_tracking", 
        fields=["tracking", "lost", "noNewCommand"],
        start=time_start,
        end=time_end)

    # Add the column of time difference in seconds
    add_column_time_differece(data_controller_state, time_start)
    add_column_time_differece(data_in_position, time_start)
    add_column_time_differece(data_tracking, time_start)

    return data_controller_state, data_in_position, data_tracking

In [ ]:
def add_column_time_differece(data_frame, time_start):
    """
    Add the column of time difference in seconds.

    Parameters
    ----------
    data_frame : pandas.core.frame.DataFrame
        Data frame.
    time_start : astropy.time.core.Time
        Start time.
    """
    if (len(data_frame.values) != 0):
        timestamp_start = pd.Timestamp(time_start.value, tz="utc")
        diff_time = data_frame.index - pd.Timestamp(time_start.value, tz="utc")
        diff_time_sec = [element.seconds + element.microseconds * 1e-6 for element in diff_time]
        data_frame["time_difference"] = diff_time_sec

In [ ]:
def plot_event_in_position(data_path_generator, time_operation, data_in_position):
    """
    Plot the inPosition events.
    
    The vertical green line means the inPosition=True, otherwise the vertical red line.
    
    Parameters
    ----------
    data_path_generator : pandas.core.frame.DataFrame
        Data of the path generator.
    time_operation : numpy.ndarray
        Operation time.
    data_in_position : pandas.core.frame.DataFrame
        Data of the inPosition event.
    """

    plt.figure()
    plt.plot(time_operation, data.demandPosition)
    plt.plot(time_operation, data_path_generator.actualPosition, "x-")
    
    # Plot the inPosition event
    for inPosition, time_inPosition in zip(data_in_position.inPosition, data_in_position.time_difference):
        if inPosition:
            plt.axvline(x=time_inPosition, color="g")
        else:
            plt.axvline(x=time_inPosition, color="r")

    plt.xlabel("Time (s)")
    plt.ylabel("P (deg)")

In [ ]:
def plot_event_tracking(data_path_generator, time_operation, data_tracking):
    """
    Plot the tracking events.
    
    The vertical yellow line means the tracking=True.
    The vertical cyan line means the tracking=False.
    The vertical black line means the noNewCommand=True.
    
    Parameters
    ----------
    data_path_generator : pandas.core.frame.DataFrame
        Data of the path generator.
    time_operation : numpy.ndarray
        Operation time.
    data_tracking : pandas.core.frame.DataFrame
        Data of the tracking event.
    """

    plt.figure()
    plt.plot(time_operation, data.demandPosition)
    plt.plot(time_operation, data_path_generator.actualPosition, "x-")
    
    # Plot the tracking event
    for tracking, noNewCommand, time_happen in zip(data_tracking.tracking, data_tracking.noNewCommand, data_tracking.time_difference):
        if tracking:
            plt.axvline(x=time_happen, color="y")
        else:
            plt.axvline(x=time_happen, color="c")

        if noNewCommand:
            plt.axvline(x=time_happen, color="k")

    plt.xlabel("Time (s)")
    plt.ylabel("P (deg)")

**Instantiate the EFD**

In [ ]:
efd_client = retrieve_efd_client(is_summit=True)

**Baseline**

Note that we need to add **4** hours more manually to query EFD data when checking the data from Chronograf.
For example, if I need a data at `2022-05-20T12:54:00`, the `Time("2022-05-20T16:54:00", scale="utc", format="isot")` is used.

In [ ]:
time_start = Time("2022-06-23T13:48:44", scale="utc", format="isot")
time_end = Time("2022-06-23T15:10:34", scale="utc", format="isot")

In [ ]:
data, acceleration, jerk, time_operation = await get_data_path_generator(efd_client, time_start, time_end)

In [ ]:
plot_path_generator(data, acceleration, jerk, time_operation)

In [ ]:
plt.figure()
plt.plot(time_operation, data.actualPosition)
plt.title("Rotator Position")
plt.xlabel("Time (sec)")
plt.ylabel("Position (deg)")

Check the data in a smaller range.

In [ ]:
time_start = Time("2022-06-23T13:48:50", scale="utc", format="isot")
time_end = Time("2022-06-23T13:49:30", scale="utc", format="isot")

In [ ]:
data_path, acceleration, jerk, time_operation = await get_data_path_generator(efd_client, time_start, time_end)
data_motor = await get_data_motors(efd_client, time_start, time_end)

In [ ]:
plot_path_generator(data_path, acceleration, jerk, time_operation)

In [ ]:
plot_currents(data_motor, time_operation)

In [ ]:
analyze_tracking_target_single(data_path, time_operation, 10, 30, show_figure=True)

In [ ]:
calculate_frequency(data_path, time_operation, 10, 30, 0, 0.0042)

In [ ]:
data_track = await get_command_data(efd_client, time_start, time_end)

In [ ]:
data_track

Check the tracking command

In [ ]:
time_check_start = 3.5
time_check_end = 12
indices_check = np.where(np.logical_and(time_operation>=time_check_start, time_operation<=time_check_end))[0]

plt.figure()
plt.plot(time_operation[indices_check], data_path.demandPosition[indices_check])
plt.xlabel("Time (sec)")
plt.ylabel("P (deg)")

# Plot the tracking command
for angle, time_track in zip(data_track.angle, data_track.time_difference):
    if (time_check_start <= time_track <= time_check_end) and (angle != 0):
        plt.axvline(x=time_track, color="r")

In [ ]:
plt.close("all")

Analyze all the data

In [ ]:
time_start = Time("2022-06-02T15:03:49", scale="utc", format="isot")
time_end = Time("2022-06-02T15:14:20", scale="utc", format="isot")
data_path, acceleration, jerk, time_operation = await get_data_path_generator(efd_client, time_start, time_end)

list_time_start = [8.5, 67.045, 126, 202, 264, 319, 377, 436, 512, 575]
list_time_end = [18.45, 74.39, 147, 208, 271, 329, 385, 457, 518, 581]
analyze_tracking_targets(data_path, time_operation, list_time_start, list_time_end)